In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import matsim
import networkx as nx

# Utils

In [3]:
''' Convert network to graph '''
def getNodeInfo(nodesDict,
                nodeCoords,
                ):
    nodesDictKeys = list(nodesDict.keys())
    nodesDictValues = list(nodesDict.values())

    node_id = None
    if nodesDict == {}:
        return node_id

    # If the node is in the nodeDict
    if nodeCoords in nodesDictValues:
        node_id = nodesDictKeys[nodesDictValues.index(nodeCoords)]

    else:  # Find the 'same' node in the nodeDict (distance < 0.2m)
        dist_array = haversine_vector(np.array(nodesDictValues),
                                      np.array([nodeCoords]),
                                      Unit.METERS,
                                      comb=True)
        # Get the index of the 'same' node (if applicable)
        try:
            closeNode_idx = np.where(dist_array < 0.2)[1][0]
        except:
            closeNode_idx = None

        # If the point is close to an existing node, return the node id of the existing node
        if closeNode_idx is not None:
            node_id = nodesDictKeys[closeNode_idx]

    return node_id

def create_graph(road_gdf: gpd.GeoDataFrame,
                 length_col='length',
                 
                 ):
    splitG = nx.DiGraph()

    # Create a dictionary to store the unique nodes (id, coords)
    nodesDict = {}

    start_nodes = []
    end_nodes = []

    node_idx = 0
    for idx, row in road_gdf.iterrows():
        # idx = row.index
        length = row[length_col]
        line = row['geometry']
        line_orig_id = row['link_id']

        # (lat, lon) format
        lineStartCoords = (line.coords[0][1], line.coords[0][0])
        lineEndCoords = (line.coords[-1][1], line.coords[-1][0])

        # Get the node id of the start and end nodes
        startNodeOrigId = row['from_node']
        endNodeOrigId = row['to_node']

        startNode_id = getNodeInfo(nodesDict, lineStartCoords)
        if startNode_id is None:
            startNode_id = node_idx
            node_idx += 1
            nodesDict[startNode_id] = lineStartCoords

        endNode_id = getNodeInfo(nodesDict, lineEndCoords)
        if endNode_id is None:
            endNode_id = node_idx
            node_idx += 1
            nodesDict[endNode_id] = lineEndCoords

        start_nodes.append(startNode_id)
        end_nodes.append(endNode_id)

        # Add nodes to the graph
        if not splitG.has_node(startNode_id):
            splitG.add_node(startNode_id, x=lineStartCoords[1], y=lineStartCoords[0], 
                            orig_id=startNodeOrigId # Add the original node id of the node
                            )
        if not splitG.has_node(endNode_id):
            splitG.add_node(endNode_id, x=lineEndCoords[1], y=lineEndCoords[0],
                            orig_id=endNodeOrigId # Add the original node id of the node
                            )
        splitG.add_edge(startNode_id, endNode_id,
                        length=length,
                        orig_id=line_orig_id,
                        idx=idx)

    splitG.graph['crs'] = "EPSG:4326"

    indexed_road = road_gdf.copy()

    indexed_road['start_node'] = start_nodes
    indexed_road['end_node'] = end_nodes

    return splitG, indexed_road

# Read the car and bike network

In [ ]:
network_root_path = r'../../../../data/intermediate/test/freightEmissions/' 

bike_network = matsim.read_network(network_root_path + 'bikeGemeenteLeuvenWithHbefaType.xml.gz')
car_network = matsim.read_network(network_root_path + 'carGemeenteLeuvenWithHbefaType.xml.gz')
raw_network = matsim.read_network(network_root_path + 'GemeenteLeuvenWithHbefaType.xml.gz')

bike_network_gdf = bike_network.as_geo()
car_network_gdf = car_network.as_geo()
raw_network_gdf = raw_network.as_geo()

bike_network_gdf.crs = 'epsg:31370'
car_network_gdf.crs = 'epsg:31370'
raw_network_gdf.crs = 'epsg:31370'

bike_nodes = bike_network.nodes
car_nodes = car_network.nodes
raw_nodes = raw_network.nodes

bike_nodes_gdf = gpd.GeoDataFrame(bike_nodes, geometry=gpd.points_from_xy(bike_nodes.x, bike_nodes.y))
bike_nodes_gdf.crs = 'epsg:31370'
car_nodes_gdf = gpd.GeoDataFrame(car_nodes, geometry=gpd.points_from_xy(car_nodes.x, car_nodes.y))
car_nodes_gdf.crs = 'epsg:31370'
raw_nodes_gdf = gpd.GeoDataFrame(raw_nodes, geometry=gpd.points_from_xy(raw_nodes.x, raw_nodes.y))
raw_nodes_gdf.crs = 'epsg:31370'

In [ ]:
bike_splitG, bike_indexed_road = create_graph(bike_network_gdf.to_crs('EPSG:4326'))
car_splitG, car_indexed_road = create_graph(car_network_gdf.to_crs('EPSG:4326'))

# Generate demand points randomly

In [6]:
bound_area = (172161.734, 174265.856, 173412.828, 175444.093)
num_points = 200

1. Generate 200 points in the `raw_network` nodes
   1. Get the coords of all demand points, and convert to `GDF`
   2. Put the GDF into `bike_gdf_dict` and `car_gdf_dict` (`{iter: gdf}`)
2. Use `nx.shortest_path` to check accessibility of each demand point at *bike_network* and *car_network*, respectively
3. Use `haversine` (or use walk network to cal the walking distance )to calculate the **extra_distance** needed for inaccessible points
4. Label the `access` and `extra_distance` of each point in the `GDF`
5. Spatial Join to get the statistics of *inaccessible num/ratio* and `extra distance`